# 03. 메타 클래스

클래스를 만드는 클래스다.

프레임워크나 모듈을 설계할 때 동적으로 클래스를 만들 수도 있을 것이다.

## - type을 사용하여 동적으로 클래스 생성

클래스 = `type("클래스이름", Tuple:부모클래스, Dict:속성과 메서드)`

```python
Calc = type("Calculator", (), {})
```

In [1]:
property_and_method = dict(
    name = "계산기",
    add = lambda self, x, y : x + y,
    mul = lambda self, x, y : x * y
)

Calc = type("Calculator", (), property_and_method)

위 클래스는 아래와 같다.

```python
class Calculator:
    name = "계산기"
    
    def add(self, x, y):
        return x + y
    
    def mul(self, x, y):
        return x * y
```

In [2]:
Calc

__main__.Calculator

In [3]:
help(Calc)

Help on class Calculator in module __main__:

class Calculator(builtins.object)
 |  Methods defined here:
 |  
 |  add lambda self, x, y
 |  
 |  mul lambda self, x, y
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)
 |  
 |  ----------------------------------------------------------------------
 |  Data and other attributes defined here:
 |  
 |  name = '계산기'



In [4]:
Calc().add(10, 20)

30

In [5]:
Calc().mul(10,20)

200

아래는 파이썬 빌트인 클래스인 `list`를 상속받아 기존의 `list` 기능 말고도 우리가 원하는 기능을 더 추가해본 코드다

In [6]:
mylist = type("MyList", (list,), {
    "info" : lambda self : print(self),
    "even" : lambda self : [num for num in self if num % 2 == 0]
})
added_list = mylist([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
added_list.info()
added_list.even()

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]


[2, 4, 6, 8, 10]

## -  `__new__`

지금까지 클래스의 인스턴스를 생성할 때 `__init__`을 통해 생성해봤기 때문에 

`__init__` 함수가 인스턴스를 생성하는 함수일 것이라 생각할 수도 있지만,

사실은 `__new__` 함수가 인스턴스를 생성하는 함수다.

위 함수는 `__init__`보다 먼저 호출되는 함수이며,

`__new__`를 이용해서 인스턴스를 먼저 생성한 다음 `__init__`을 통해 인스턴스에 값을 넣는다.

In [7]:
class Student:
    def __new__(cls):
        print('__new__가 호출되었습니다')
    
    def __init__(self):
        print('__init__가 호출되었습니다')
    
    def study(self):
        print("A student is studying")

In [8]:
stu = Student()

__new__가 호출되었습니다


위 결과를 보면 알겠지만 `Student` 클래스의 인스턴스가 생성될 때 `__init__`이 아닌 `__new__` 함수만 호출된 것을 볼 수 있다.

사실 `__init__`함수가 호출되지 않은 것은 우리가 `__new__` 함수를 오버라이딩해버려서 `__init__`을 통해 인스턴스 값을 못넘겨받았기 때문이다.

In [9]:
stu

In [10]:
stu.study()

AttributeError: 'NoneType' object has no attribute 'study'

In [11]:
type(stu)

NoneType

위 코드를 보면 알 수 있듯이 현재 `__new__`가 실행되어 인스턴스는 생성되었지만 인스턴스에 대한 정보가 아무것도 없다.

즉, 정리하자면 `__new__`는 해당 클래스의 객체(or 인스턴스)를 메모리에 할당을 해주고

클래스의 정보를 객체에 작성해주는 녀석은 `__init__`인 것이다.

이제 제대로 구현해보자.

In [12]:
class Student:
    def __new__(cls):
        print('__new__가 호출되었습니다')
        student = object.__new__(cls) # 최상위 부모 클래스인 object를 통하여 객체 생성
        return student # 생성된 객체를 __init__에 넘겨준다.
    
    def __init__(self):
        print('__init__가 호출되었습니다')
    
    def study(self):
        print("A student is studying")

In [13]:
stu = Student()

__new__가 호출되었습니다
__init__가 호출되었습니다


In [14]:
stu

In [15]:
stu.study()

A student is studying


In [16]:
type(stu)

__main__.Student

 - `__new__`에 대한 오버라이딩의 목적은 클래스를 커스텀마이징하거나 클래스 생성 등에 대한 핸들링 등이 있다
 
아래 클래스는 3개의 객체만 생성할 수 있게 제한을 걸어둔 클래스다.

아래와 같은 기능이 필요한 때가 있다. 

서버와 데이터베이스를 연결한 connection 객체를 관리해주는 connection pool이라는 객체가 여러 개 있으면 안될 것이다.

그래서 아래와 같은 방법으로 단 1개의 객체만 생성하도록 제한하는 하는 것을 사용한다.

이를 프로그래밍 디자인패턴 기법 중 하나인 `싱글톤 패턴(singleton pattern)`이라고 한다.

In [17]:
class LimitedInstances():
    _instances = []  # Keep track of instance reference
    limit = 3
    def __new__(cls, *args, **kwargs):
        print("---------인스턴스 생성 전 클래스 정보 확인---------")
        print("| 제한된 객체의 수 : ", cls.limit)
        print("| 현재 생성된 객체 리스트 : ", cls._instances)
        if not len(cls._instances) < cls.limit:
            raise RuntimeError("Count not create instance. Limit {} reached".format(cls.limit)) # 강제 에러 발생
        instance = object.__new__(cls, *args, **kwargs)
        cls._instances.append(instance)
        print("--------------------------------------------------")
        return instance
    
    def __del__(self):
        # Remove instance from _instances
        self._instance.remove(self)

a = LimitedInstances()
LimitedInstances()
LimitedInstances()
LimitedInstances()

---------인스턴스 생성 전 클래스 정보 확인---------
| 제한된 객체의 수 :  3
| 현재 생성된 객체 리스트 :  []
--------------------------------------------------
---------인스턴스 생성 전 클래스 정보 확인---------
| 제한된 객체의 수 :  3
| 현재 생성된 객체 리스트 :  [<__main__.LimitedInstances object at 0x000001450C4C6490>]
--------------------------------------------------
---------인스턴스 생성 전 클래스 정보 확인---------
| 제한된 객체의 수 :  3
| 현재 생성된 객체 리스트 :  [<__main__.LimitedInstances object at 0x000001450C4C6490>, <__main__.LimitedInstances object at 0x000001450C3F3850>]
--------------------------------------------------
---------인스턴스 생성 전 클래스 정보 확인---------
| 제한된 객체의 수 :  3
| 현재 생성된 객체 리스트 :  [<__main__.LimitedInstances object at 0x000001450C4C6490>, <__main__.LimitedInstances object at 0x000001450C3F3850>, <__main__.LimitedInstances object at 0x000001450C409F40>]


RuntimeError: Count not create instance. Limit 3 reached

In [18]:
a.limit

3

In [19]:
a._instances #_instances라고 선언해야됨

## - `__call__`

`__new__`가 클래스 인스턴스를 생성해 메모리에 할당하고

`__init__`이 인스턴스에 값들을 넣고 초기화하한다면

`__call__`함수는 인스턴스 자체가 호출될 때 호출되는 함수다.

`__new__` ==> `__init__` ==> `__call__`

In [20]:
class MyClass:
    def __init__(self,msg):
        self.__msg = msg
        
    def __call__(self):
        print(len(self.__msg))
        return [s for s in self.__msg]
    
mine = MyClass("Hello World")
mine() # 인스턴스 자체를 호출

11


['H', 'e', 'l', 'l', 'o', ' ', 'W', 'o', 'r', 'l', 'd']

## - type을 상속받아 클래스를 만드는 클래스인 메타클래스가 구현하는 방식을 정하기

```python
class MetaClassName(type):
    def __new__(metacls, name, bases, namespace):
        #Statement
```

In [21]:
class MakeCalc(type):
    def __new__(matacls, name, bases, namespace):
        namespace["add"] = lambda self, *args : sum(args)
        return type.__new__(matacls, name, bases, namespace)

Calc = MakeCalc("Calculator",(),{})
c = Calc()
print(c.add(1,2,3,4,5,6,7,8,9,10))

55


 - 활용 CASE : 싱글톤(Singleton)

In [22]:
class Singleton(type):
    __instances = {}
    def __call__(cls, *args, **kwargs):
        if cls not in cls.__instances:
            cls.__instances[cls] = super().__call__(*args, **kwargs)
        return cls.__instances[cls]
    
class Hello(metaclass=Singleton): # 메타클래스로 Singleton을 지정
    def __init__(self):
        self.desc = "Hello"

a = Hello()
b = Hello()
print(a is b) # 주소가 같음. 같은 객체임

print(a.desc, ":" ,b.desc)
a.desc = "Hello World"
print(a.desc, ":" ,b.desc)

True
Hello : Hello
Hello World : Hello World


 - 메타클래스와 new, init, call의 관계를 정리하자면 아래와 같다
 - `obj` == `MyClass()` == `(MyMetaClass())()`

In [23]:
class MyMetaClass(type):
    def __new__(cls, *args, **kwargs):
        print("MyMetaClass   __new__")
        return super().__new__(cls,*args,**kwargs)
    
    def __init__(cls, *args, **kwargs):
        print("MyMetaClass   __init__")
        return super().__init__(*args,**kwargs)  
    
    def __call__(cls, *args, **kwargs):
        print("MyMetaClass   __call__")
        return super().__call__(*args,**kwargs)
    
class MyClass(metaclass=MyMetaClass):
    def __new__(cls,*args, **kwargs):
        print('__new__가 호출되었습니다')
        mycls = object.__new__(cls,*args, **kwargs)
        return mycls
    
    def __init__(self):
        print('__init__가 호출되었습니다')
    
    def __call__(self):
        print('__call__가 호출되었습니다')
        
    def hello(self):
        print("hello world")
        
print("="*20)
obj = MyClass()
obj.hello()
obj()

MyMetaClass   __new__
MyMetaClass   __init__
MyMetaClass   __call__
__new__가 호출되었습니다
__init__가 호출되었습니다
hello world
__call__가 호출되었습니다


<br>
<br>
<br>
<br>
<br>
<br>
<hr>
<br>
<br>
<br>
<br>
<br>
<br>